In [4]:
import tensorflow as tf

In [1]:
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [5]:
batch_size = 128
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, n_classes])

In [7]:
def conv2d(x, W):
    return tf.nn.conv2d(x,W, strides=[1,1,1,1], padding='SAME')

Reason why we use strides = [1 stride stride 1]
https://stackoverflow.com/questions/34642595/tensorflow-strides-argument?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa

In [8]:
def maxpool2d(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [9]:
def convolutional_neural_network(x):
    weights = {
        'W_conv1': tf.Variable(tf.random_normal([5,5,1,32])),
        'W_conv2': tf.Variable(tf.random_normal([5,5,32,64])),
        'W_fc': tf.Variable(tf.random_normal([7*7*64, 1024])),
        'out': tf.Variable(tf.random_normal([1024, n_classes]))
    }
    
    biases = {
        'b_conv1': tf.Variable(tf.random_normal([32])),
        'b_conv2': tf.Variable(tf.random_normal([64])),
        'b_fc': tf.Variable(tf.random_normal([1024])),
        'out': tf.Variable(tf.random_normal([n_classes])),
    }
    #reshape input to a 4D tensor
    x = tf.reshape(x, shape=[-1,28,28,1])
    
    conv1 = tf.nn.relu(conv2d(x, weights['W_conv1'])+biases['b_conv1'])
    conv1 = maxpool2d(conv1)
    
    conv2 = tf.nn.relu(conv2d(conv1, weights['W_conv2'])+biases['b_conv2'])
    conv2 = maxpool2d(conv2)
    
    fc = tf.reshape(conv2, [-1,7*7*64])
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
    
    output = tf.matmul(fc, weights['out'])+biases['out']
    return output
    

In [10]:

def train_neural_network(x):
    prediction = convolutional_neural_network(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction, labels=y) )
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    hm_epochs = 10
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c

            print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))

In [11]:
train_neural_network(x)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Epoch 0 completed out of 10 loss: 1331079.3392333984
Epoch 1 completed out of 10 loss: 272082.85095214844
Epoch 2 completed out of 10 loss: 168399.89845570177
Epoch 3 completed out of 10 loss: 111377.4054851532
Epoch 4 completed out of 10 loss: 86163.50411987305
Epoch 5 completed out of 10 loss: 59394.575382232666
Epoch 6 completed out of 10 loss: 45661.6176186949
Epoch 7 completed out of 10 loss: 32889.26181362188
Epoch 8 completed out of 10 loss: 29378.34655714687
Epoch 9 completed out of 10 loss: 21753.061862945557
Accuracy: 0.9734


## TFLearn 
lets use TFLearn for faster and less error pron development

In [14]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

In [15]:
import tflearn.datasets.mnist as mnist

X, Y, test_x, test_y = mnist.load_data(one_hot=True)


Succesfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting mnist/train-images-idx3-ubyte.gz
Succesfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting mnist/train-labels-idx1-ubyte.gz
Succesfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting mnist/t10k-images-idx3-ubyte.gz
Succesfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting mnist/t10k-labels-idx1-ubyte.gz


In [16]:
X = X.reshape([-1, 28, 28, 1])
test_x = test_x.reshape([-1, 28, 28, 1])

In [17]:
convnet = input_data(shape=[None, 28, 28, 1], name='input')

In [18]:
convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


In [19]:
convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

In [20]:
convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

In [21]:
convnet = fully_connected(convnet, 10, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=0.01, loss='categorical_crossentropy', name='targets')

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [22]:
model = tflearn.DNN(convnet)

In [23]:
model.fit({'input': X}, {'targets': Y}, n_epoch=10, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=500, show_metric=True, run_id='mnist')

Training Step: 8599  | total loss: 0.27127 | time: 60.627s
| Adam | epoch: 010 | loss: 0.27127 - acc: 0.9675 -- iter: 54976/55000
Training Step: 8600  | total loss: 0.25171 | time: 63.023s
| Adam | epoch: 010 | loss: 0.25171 - acc: 0.9692 | val_loss: 0.07770 - val_acc: 0.9790 -- iter: 55000/55000
--


In [24]:
model.save('quicktest.model')

INFO:tensorflow:/home/dheeraj/work/sentdex/ml/datascience/Machine Learning/quicktest.model is not in all_model_checkpoint_paths. Manually adding it.


In [25]:
import numpy as np
print( np.round(model.predict([test_x[1]])[0]) )
print(test_y[1])

[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
